In [1]:
# !pip install xmltodict tqdm --quiet
import os
import shutil
import xmltodict
from tqdm import tqdm
from pathlib import Path
import yaml
import PIL.Image as Image

In [2]:
# 1. Define class
FGVD_PATH = "/data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/IDD_FGVD"  # Thư mục gốc chứa train/val/test
YOLO_PATH = "/data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8"
ALLOWED_CLASSES = {"car": 0, "motorcycle": 1}

In [3]:
# 2. Create structure folder for YOLOv8
for split in ["train", "val"]:
    for sub in ["images", "labels"]:
        os.makedirs(os.path.join(YOLO_PATH, sub, split), exist_ok=True)

In [4]:
# 3. Function convert XML to YOLO
def convert_voc_to_yolo(xml_path, img_w, img_h):
    with open(xml_path, "r") as f:
        data = xmltodict.parse(f.read())

    yolo_lines = []
    objects = data["annotation"].get("object", [])
    if not isinstance(objects, list):
        objects = [objects]

    for obj in objects:
        name = obj["name"].split("_")[0]
        if name not in ALLOWED_CLASSES:
            continue
        cls_id = ALLOWED_CLASSES[name]
        bbox = obj["bndbox"]
        xmin = float(bbox["xmin"])
        ymin = float(bbox["ymin"])
        xmax = float(bbox["xmax"])
        ymax = float(bbox["ymax"])
        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h
        yolo_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    return yolo_lines

In [5]:
# 4. Processing data train/val
for split in ["train", "val"]:
    image_dir = Path(FGVD_PATH) / split / "images"
    label_dir = Path(FGVD_PATH) / split / "annos"
    yolo_img_dir = Path(YOLO_PATH) / "images" / split
    yolo_lbl_dir = Path(YOLO_PATH) / "labels" / split

    for image_path in tqdm(list(image_dir.glob("*.jpg")), desc=f"Processing {split}"):
        xml_file = label_dir / f"{image_path.stem}.xml"
        if not xml_file.exists():
            continue
        try:
            with Image.open(image_path) as img:
                w, h = img.size
        except:
            continue
        yolo_lines = convert_voc_to_yolo(xml_file, w, h)
        if len(yolo_lines) == 0:
            continue
        shutil.copy(image_path, yolo_img_dir / image_path.name)
        with open(yolo_lbl_dir / f"{image_path.stem}.txt", "w") as f:
            f.write("\n".join(yolo_lines))

print("FGVD converted to format YOLO !!!")

Processing val: 100%|██████████| 884/884 [00:01<00:00, 827.39it/s]

FGVD converted to format YOLO !!!


In [6]:
# 5. create file fgvd.yaml
yaml_path = Path(YOLO_PATH) / "fgvd.yaml"
yaml_content = {
    "path": YOLO_PATH,
    "train": "images/train",
    "val": "images/val",
    "nc": 2,
    "names": ["car", "motorcycle"]
}
with open(yaml_path, "w") as f:
    yaml.dump(yaml_content, f)

print(f"Created file {yaml_path} YOLOv8.")

Created file /data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/fgvd.yaml YOLOv8.


In [8]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
# model = YOLO("yolo11n.pt")
model.train(
    data="/data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/fgvd.yaml", 
    epochs=100, 
    imgsz=640, 
    batch=16,
    lr0=0.01,
    patience=10,
    optimizer="AdamW"
)

New https://pypi.org/project/ultralytics/8.3.161 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.152 🚀 Python-3.12.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 14918MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/fgvd.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=tr

train: Scanning /data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/labels/train... 3204 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3204/3204 [00:03<00:00, 806.76it/s]

train: New cache created: /data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/labels/train.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1776.3±1050.7 MB/s, size: 530.3 KB)


val: Scanning /data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/labels/val... 797 images, 0 backgrounds, 0 corrupt: 100%|██████████| 797/797 [00:02<00:00, 278.88it/s]

...

                   all        797       2058       0.83      0.816      0.883        0.7
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 86, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



96 epochs completed in 0.950 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.12.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 14918MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.36it/s]


                   all        797       2058      0.841      0.803      0.883        0.7
                   car        618       1278      0.878      0.844      0.926      0.795
            motorcycle        474        780      0.805      0.762       0.84      0.605
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f374b1206b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [9]:
# Measures accuracy of trained model on validation data

model = YOLO("/data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/runs/detect/train/weights/best.pt")
metrics = model.val()

print(f"mAP@.5:.95 = {metrics.box.map:.4f}")
print(f"mAP@.50    = {metrics.box.map50:.4f}")
print(f"mAP@.75    = {metrics.box.map75:.4f}")

Ultralytics 8.3.152 🚀 Python-3.12.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 14918MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3566.9±255.1 MB/s, size: 511.2 KB)


val: Scanning /data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/labels/val.cache... 797 images, 0 backgrounds, 0 corrupt: 100%|██████████| 797/797 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:06<00:00,  7.70it/s]


                   all        797       2058      0.841      0.803      0.883      0.701
                   car        618       1278      0.877      0.844      0.926      0.796
            motorcycle        474        780      0.805      0.762      0.841      0.606
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val
mAP@.5:.95 = 0.7006
mAP@.50    = 0.8834
mAP@.75    = 0.7936


In [10]:
# Benchmark for model fine-tuning
from ultralytics.utils.benchmarks import benchmark

model.benchmark(
    model="/data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/runs/detect/train/weights/best.pt",
    data="/data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/fgvd.yaml",
    imgsz=640,
    half=False,
    device="cuda"
)

Setup complete ✅ (16 CPUs, 125.8 GB RAM, 66.6/95.9 GB disk)

Benchmarks complete for /data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/runs/detect/train/weights/best.pt on /data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/fgvd.yaml at imgsz=640 (513.26s)
Benchmarks legend:  - ✅ Success  - ❎ Export passed but validation failed  - ❌️ Export failed
                   Format Status❔  Size (MB) metrics/mAP50-95(B) Inference time (ms/im)     FPS
0                 PyTorch       ✅        5.9              0.7006                   9.23   108.3
1             TorchScript       ✅       11.9              0.6937                   4.62  216.26
2                    ONNX       ✅       11.7              0.6937                   7.33  136.37
3                OpenVINO       ❌        0.0                   -                      -       -
4                TensorRT       ✅       15.6              0.6937                    3.7  269.92
5                  CoreML       ❌        0.0          

,Format,Status❔,Size (MB),metrics/mAP50-95(B),Inference time (ms/im),FPS
0,PyTorch,✅,5.9,0.7006,9.23,108.30
1,TorchScript,✅,11.9,0.6937,4.62,216.26
2,ONNX,✅,11.7,0.6937,7.33,136.37
3,OpenVINO,❌,0.0,NaN,NaN,NaN
4,TensorRT,✅,15.6,0.6937,3.70,269.92
5,CoreML,❌,0.0,NaN,NaN,NaN
6,TensorFlow SavedModel,❎,29.4,NaN,NaN,NaN
7,TensorFlow GraphDef,❎,11.8,NaN,NaN,NaN
8,TensorFlow Lite,❌,0.0,NaN,NaN,NaN
9,TensorFlow Edge TPU,❌,0.0,NaN,NaN,NaN


In [5]:
# Benchmark for model fine-tuning
from ultralytics import YOLO
from ultralytics.utils.benchmarks import benchmark

model = YOLO("yolov8n.pt")

metrics = model.val(
    data="/data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/fgvd.yaml", 
    # epochs=100, 
    imgsz=640, 
    batch=16
    # lr0=0.01,
    # patience=10,
    # optimizer="AdamW"
)
# print(metrics)
print(f"mAP@.5:.95 = {metrics.box.map:.4f}")
print(f"mAP@.50    = {metrics.box.map50:.4f}")
print(f"mAP@.75    = {metrics.box.map75:.4f}")

Ultralytics 8.3.152 🚀 Python-3.12.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 14918MiB)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3907.2±419.4 MB/s, size: 464.3 KB)


val: Scanning /data/jupyter/dungnd/IT3320E-DeepLearning/capstone_prj/FGVD_YOLOv8/labels/val.cache... 797 images, 0 backgrounds, 0 corrupt: 100%|██████████| 797/797 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:07<00:00,  6.85it/s]


                   all        797       2058     0.0352      0.389     0.0597     0.0247
                person        618       1278    0.00197     0.0962    0.00109   0.000532
               bicycle        474        780     0.0685      0.682      0.118     0.0489
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/val27
mAP@.5:.95 = 0.0247
mAP@.50    = 0.0597
mAP@.75    = 0.0162
